In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import datasets, transforms


from SupConLosszy import SupConLoss
import copy


manualSeed = 999
print("Random Seed: ", manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

torch.cuda.manual_seed_all(manualSeed)
torch.backends.cudnn.deterministic = True

Random Seed:  999


In [2]:

#Fourier transform

import librosa

def changesy(xtrain0):

    sn=np.array(xtrain0)
    sy=xtrain0.unsqueeze(1)
    X = librosa.stft(sn, n_fft=52, hop_length=64)
    a=np.abs(X)
    b=np.angle(X)
    c=[]
    c.append(a)
    c.append(b)
    c.append(sy)
    c=np.concatenate(c)
    
    return c
    

def changeX(xtrain):
    yt=[]
    countx=len(xtrain)
    print("len:",countx)
    for i in range(countx):
        yt.append(changesy(xtrain[i]))
        qt=np.array(yt)
        qt=torch.tensor(qt)
        qt=torch.squeeze(qt)
    return qt
        

In [3]:

print('==> Preparing data.............................')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets import MNIST, CIFAR10, CIFAR100, SVHN

import os
import torch
from torch import nn,optim
import torch.nn.functional as F

from torchvision import datasets, transforms

from time import perf_counter

import  numpy as np
import torch.utils.data as Data

from torch.utils.data import Dataset, DataLoader

class Safeman(Dataset):
    
    def __init__(self, data,targets):
        super(Safeman, self).__init__()
        self.data = data
        self.targets = targets
        
     
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        return img, target

class Safeman_Filter(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        for i in range(len(targets)):
            if targets[i] in known:
                mask.append(i)
                new_targets.append(known.index(targets[i]))
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)
        
class Safeman_FilterB(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        new_targets = []
        for i in range(len(targets)):
            if targets[i] in known:
                new_targets.append(0)
            else:
                new_targets.append(1)
        self.targets = np.array(new_targets)
        self.data = self.data

class Safeman_FilterC(Safeman):
    
    def __Filter__(self, trainknown):
        train_class_num=len(trainknown)
        for i in range(0,len(self.targets)) :
            if self.targets[i]>train_class_num:
                self.targets[i] = train_class_num
        self.data = self.data

class Safeman_FilterD(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        train_class_num=len(known)
        for i in range(len(targets)):
            if targets[i] not in known:
                mask.append(i)
                new_targets.append(train_class_num)
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)



known=[0, 1, 2,3,4]
unknown=[ 5,6,7]

num_class=len(known)



X_train0 = np.load('./dataset/X_train_ukm.npy')
y_train1 = np.load('./dataset/y_train_ukm.npy')
X_final_test0 = np.load('./dataset/X_final_test_ukm.npy' )
y__final_test1 = np.load('./dataset/y__final_test_ukm.npy')



print(X_train0.shape, X_final_test0.shape, y_train1.shape,y__final_test1.shape)



x_train11=changeX(torch.tensor(X_train0))

x_test11=changeX(torch.tensor(X_final_test0))

X_train00=x_train11
X_final_test00=x_test11

print(X_train00.shape, X_final_test00.shape, y_train1.shape,y__final_test1.shape)

X_train1=[]
X_final_test1=[]

for i in range(len(y_train1)):
    a = np.resize(X_train00[i], (1, 28, 28))
    X_train1 += [a]
    
for j in range(len(y__final_test1)):
    b = np.resize(X_final_test00[j], (1, 28, 28))
    X_final_test1 += [b]

i=0
j=0



x_train, x_test, y_train,y_test = torch.Tensor(X_train1), torch.Tensor(X_final_test1), torch.from_numpy(y_train1), torch.from_numpy(y__final_test1)

print(x_train.shape, x_test.shape, y_train.shape,y_test.shape)



train_dataset = Data.TensorDataset(x_train, y_train)
train_dataset.data = train_dataset.tensors[0]
train_dataset.targets = train_dataset.tensors[1]



test_dataset = Data.TensorDataset(x_test, y_test)
test_dataset.data = test_dataset.tensors[0]
test_dataset.targets = test_dataset.tensors[1]


labels =['ARP poisining', 'BeEF HTTP exploits','Mass HTTP requests','Metasploit exploits','Normal','Port scanning','TCP flood','UDP data flood']



train_dataset.classes = labels
test_dataset.classes = labels

train_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}
test_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}



b_s=16

trainset = Safeman_Filter(data=train_dataset.data,targets=train_dataset.targets)
print('All down Train Data:', len(trainset))
trainset.__Filter__(known=known)


train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=b_s, shuffle=True,
    num_workers=4,drop_last=True)


print('Real train Data:', len(trainset))



testsetA = Safeman_Filter(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetA.__Filter__(known=known)


test_loader_A = torch.utils.data.DataLoader(
    testsetA, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetA Data:', len(testsetA))


testsetD = Safeman_FilterD(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetD.__Filter__(known=known)


test_loader_D = torch.utils.data.DataLoader(
    testsetD, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetD Data:', len(testsetD))


testsetC = Safeman_FilterC(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetC Data:', len(testsetC))
testsetC.__Filter__(trainknown=known)


test_loader_C = torch.utils.data.DataLoader(
    testsetC, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetC Data:', len(testsetC))


print("done!")



==> Preparing data.............................
(10309, 46) (2578, 46) (10309,) (2578,)
len: 10309


/data1/wk/anaconda3/envs/cvaecaposr/lib/python3.6/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=52 is too small for input signal of length=46
  return f(*args, **kwargs)


len: 2578
torch.Size([10309, 100]) torch.Size([2578, 100]) (10309,) (2578,)
torch.Size([10309, 1, 28, 28]) torch.Size([2578, 1, 28, 28]) torch.Size([10309]) torch.Size([2578])
All down Train Data: 10309
Real train Data: 8923
All testsetA Data: 2578
Real testsetA Data: 2226
All testsetA Data: 2226
Real testsetD Data: 352
All testsetC Data: 2578
Real testsetC Data: 2578
done!


In [4]:
from torch.nn import Module
from torch import nn
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

class Modelnsl8(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64) 
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, num_class) 
    
    def forward(self, x,trainT=False):        
        x = F.relu(self.fc1(x))
        x4=x
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x3=x
        feat = F.normalize(x3,dim=1)
        feat1 = F.normalize(x4,dim=1)
        x = self.fc4(x)
        if trainT == True:
            return feat, F.log_softmax(x, dim = 1)
        else:
            return F.log_softmax(x, dim = 1)

In [5]:
train_loader2 = train_loader
test_loader2 = test_loader_A
model = Modelnsl8()
sgd = SGD(model.parameters(), lr=1e-2)
criterion = SupConLoss(temperature=0.7)
loss_fn = CrossEntropyLoss()
all_epoch = 15 

targetctA=[]
targetctD=[]


predctA=[]
predctD=[]


targettrain=[]
KresA=[]
KlossA=[]
featestA=[]
featestD=[]
KresD=[]
KlossD=[]

#Step 1: the DNN+CL model is trained and tested without adversarial training and without adversarial attacks

batch_size = b_s
for current_epoch in range(all_epoch):
    model.train()
    for idx, (train_x, train_label) in enumerate(train_loader2):
        sgd.zero_grad()
        features1,predict_y = model(train_x.float().view(-1, 28*28),trainT=True)
        
        if current_epoch==all_epoch-1:
            targettrain+=train_label
        
        features=torch.cat([features1, features1], dim=0)
        f1, f2 = torch.split(features, [batch_size, batch_size], dim=0)
        feat = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)

        
        target2=train_label
        loss2 = criterion(feat, target2)
        loss1 = loss_fn(predict_y, train_label.long())
        loss=loss1+loss2
        if idx % 1000 == 0:
            print('idx: {}, loss: {}'.format(idx, loss.sum().item()))
        loss.backward()
        sgd.step()
    print("epoch i=",current_epoch+1)
    print("this train epoch end")
    all_correct_num = 0
    all_sample_num = 0
    model.eval()
    for idx, (test_x, test_label) in enumerate(test_loader2):
        _,predict_y = model(test_x.float().view(-1, 28*28),trainT=True)
        predict_y=predict_y.detach()
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print('accuracy: {:.2f}'.format(acc))
print("training end")

#-----------------------------------------------------------------------



all_correct_num = 0
all_sample_num = 0
model.eval()
for idx, (test_x, test_label) in enumerate(test_loader2):
    feature1,predict_y = model(test_x.float().view(-1, 28*28),trainT=True)
    featestA+=feature1
    predict_y=predict_y.detach()
    KresA+=predict_y 
    predict_y = np.argmax(predict_y, axis=-1)
    targetctA+=test_label
    predctA+=predict_y
    current_correct_num = predict_y == test_label
    all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
    all_sample_num += current_correct_num.shape[0]
acc = all_correct_num / all_sample_num




all_correct_num = 0
all_sample_num = 0
model.eval()
for idx, (test_x, test_label) in enumerate(test_loader_D):
    feature1,predict_y = model(test_x.float().view(-1, 28*28),trainT=True)
    featestD+=feature1
    predict_y=predict_y.detach()
    KresD+=predict_y
    predict_y = np.argmax(predict_y, axis=-1)
    targetctD+=test_label
    predctD+=predict_y
    current_correct_num = predict_y == test_label
    all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
    all_sample_num += current_correct_num.shape[0]
acc = all_correct_num / all_sample_num


from sklearn.metrics import accuracy_score

print("test_loader_A accuracy",accuracy_score(targetctA,predctA))


tarad=targetctA+targetctD
pread=predctA+predctD

print("test_loader_A_D accuracy",accuracy_score(tarad,pread))

torch.save(targetctA,'./savedata/targetctA1225.pkl')

torch.save(tarad,'./savedata/tarad1225.pkl')
torch.save(pread,'./savedata/pread1225.pkl')

torch.save(targettrain,'./savedata/targettrain1229.pkl')




idx: 0, loss: 35.8233528137207
epoch i= 1
this train epoch end
accuracy: 0.95
idx: 0, loss: 29.944602966308594
epoch i= 2
this train epoch end
accuracy: 0.96
idx: 0, loss: 29.68160057067871
epoch i= 3
this train epoch end
accuracy: 0.97
idx: 0, loss: 31.362390518188477
epoch i= 4
this train epoch end
accuracy: 0.97
idx: 0, loss: 33.42679977416992
epoch i= 5
this train epoch end
accuracy: 0.97
idx: 0, loss: 34.75148391723633
epoch i= 6
this train epoch end
accuracy: 0.97
idx: 0, loss: 34.43937301635742
epoch i= 7
this train epoch end
accuracy: 0.97
idx: 0, loss: 33.100341796875
epoch i= 8
this train epoch end
accuracy: 0.97
idx: 0, loss: 31.851158142089844
epoch i= 9
this train epoch end
accuracy: 0.96
idx: 0, loss: 33.985679626464844
epoch i= 10
this train epoch end
accuracy: 0.97
idx: 0, loss: 32.01341247558594
epoch i= 11
this train epoch end
accuracy: 0.97
idx: 0, loss: 29.59646987915039
epoch i= 12
this train epoch end
accuracy: 0.98
idx: 0, loss: 27.718324661254883
epoch i= 13
thi

In [6]:


Kres=KresA+KresD

torch.save(Kres,'./savedata/Kres1225.pkl')

featest=featestA+featestD


Krestp=[]
for i in range(len(Kres)):
    Krestp.append(Kres[i].numpy())
    
np.savetxt(r"./savedata/Kres1225.txt",Krestp)

torch.save(featestA,'./savedata/featestA1229.pkl')

torch.save(featestD,'./savedata/featestD1229.pkl')

torch.save(featest,'./savedata/featest.pkl')


torch.save(model, './savedata/DNNimproveorigin0105fanhuidanfftd.pkl')

#Step 1 end

In [7]:
train_loader2 = train_loader
test_loader2 = test_loader_A


model=torch.load('./savedata/DNNimproveorigin0105fanhuidanfftd.pkl')

sgd = SGD(model.parameters(), lr=1e-2)
criterion = SupConLoss(temperature=0.7)


loss_fn = CrossEntropyLoss()
batch_size = b_s
optimizer=sgd
criterion3=loss_fn

import foolbox

import numpy as np
import os
import csv
import time


all_correct_num = 0
all_sample_num = 0


allepoch=3 

predAattack=[]
predAnoattack=[]
targetAnoattack=[]




testAyizhiadv=[]

featestAzyori=[]
featestAzyadv=[]

predctAzyori=[]
predctAzyadv=[]

predctDzy=[]
featestDzy=[]

featesttrain=[]



for epoch in range(allepoch):
    
    
    model.eval()

    model=model.cuda()

    test_total, test_correct, test_robustness = 0, 0, 0

    fmodel = foolbox.models.PyTorchModel(model, bounds=(-255, 255))

    attack = foolbox.attacks.LinfFastGradientAttack() #FGSM

    start_time = time.time()
    
    #Step 3 and step 4 show that our model is tested twice, once without perturbation and once with perturbation,
    # which is closer to a real scenario, i.e., not all traffic will be perturbed.
    
    for idx, (test_x, test_label) in enumerate(test_loader2):
        
        #Step 3: the DNN+CL model is trained and tested with adversarial training and without adversarial attacks(for last epoch)

        test_x=test_x.cuda()
        test_label=test_label.cuda()


        features1,predict_y = model(test_x.float().view(-1, 28*28),trainT=True)
        predict_y=predict_y.detach()
        predict_y=predict_y.cpu()
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label.cpu()
        if epoch == allepoch-1:
            predAnoattack+=predict_y
            targetAnoattack+=test_label.cpu()
            featestAzyori+=features1
            predctAzyori+=predict_y
        test_correct += np.sum(current_correct_num.numpy(), axis=-1)
        test_total += current_correct_num.shape[0]
        
        
        #Step 3 end
        
        
        #Step 4: the DNN+CL model is trained and tested with adversarial training and with adversarial attacks(for last epoch)


        _, adv_untargeted, _ = attack(fmodel, test_x.float().view(-1, 28*28), test_label, epsilons=12/255) # adversarial attack
        features1,predict_y2 = model(adv_untargeted.float().view(-1, 28*28),trainT=True)
        predict_y2=predict_y2.detach()
        
        predict_y2=predict_y2.cpu()
        predict_y2 = np.argmax(predict_y2, axis=-1)
        current_correct_num2 = predict_y2 == test_label.cpu()
        if epoch == allepoch-1:
            predAattack+=predict_y2
            testAyizhiadv+=adv_untargeted
            featestAzyadv+=features1
            predctAzyadv+=predict_y2
        test_robustness += np.sum(current_correct_num2.numpy(), axis=-1)
        #Step 4 end


    test_acc, test_adv_acc = test_correct / test_total, test_robustness / test_total

    # Record the time on the testset
    end_time = time.time()
    testset_total_time = end_time - start_time

    print("epoch: {} Time:{} clean_test_acc: {:.3f}  adv_test_acc: {:.3f}".format(epoch,testset_total_time, test_acc, test_adv_acc))

    
    train_total, train_correct, train_robustness = 0, 0, 0

    start_time = time.time()
    
    
    if epoch == allepoch-1:
        continue
        
        
    #Step 2: the DNN+CL model is trained with adversarial training

    for idx, (test_x, test_label) in enumerate(train_loader):
        

        fmodel = foolbox.models.PyTorchModel(model, bounds=(-255, 255))

        attack = foolbox.attacks.LinfFastGradientAttack()

        test_x=test_x.cuda()
        test_label=test_label.cuda()
        
        
        features1,predict_y = model(test_x.float().view(-1, 28*28),trainT=True)
        
        if epoch==allepoch-2:
            featesttrain+=features1
        
        features=torch.cat([features1, features1], dim=0)
        f1, f2 = torch.split(features, [batch_size, batch_size], dim=0)
        feat = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)


        predict_y=predict_y.detach()
        predict_y_temp=predict_y
        predict_y=predict_y.cpu()
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label.cpu()
        train_correct += np.sum(current_correct_num.numpy(), axis=-1)
        train_total += current_correct_num.shape[0]


        _, adv_untargeted, _ = attack(fmodel, test_x.float().view(-1, 28*28), test_label, epsilons=12/255) # adversarial attack
        
        features2,predict_y2 = model(adv_untargeted.float().view(-1, 28*28),trainT=True)
        features11=torch.cat([features2, features2], dim=0)
        f11, f21 = torch.split(features11, [batch_size, batch_size], dim=0)
        feat1 = torch.cat([f11.unsqueeze(1), f21.unsqueeze(1)], dim=1)
        

        
        predict_y2=predict_y2.detach()
        predict_y_temp2=predict_y2
        predict_y2=predict_y2.cpu()
        predict_y2 = np.argmax(predict_y2, axis=-1)
        current_correct_num2 = predict_y2 == test_label.cpu()
        train_robustness += np.sum(current_correct_num2.numpy(), axis=-1)
        
        
        model.train()
        adv_loss = criterion3(predict_y_temp, test_label)
        clean_loss = criterion3(predict_y_temp2, test_label)
        loss0 = criterion(feat, test_label)
        loss1 = criterion(feat1, test_label)
        sgd.zero_grad()
        loss = clean_loss + adv_loss+loss0+loss1
        loss.requires_grad_()
        loss.backward()

        sgd.step()
        model.eval()


    train_acc, train_adv_acc = train_correct / train_total, train_robustness / train_total
    
    #Step 2 end
    
    model.eval()

    # Record the time on the testset
    end_time = time.time()
    trainset_total_time = end_time - start_time

    print("----epoch: {} Time:{} clean_train_acc: {:.3f}  adv_train_acc: {:.3f}".format(epoch, trainset_total_time, train_acc, train_adv_acc))
    

torch.save(model, './savedata/DNNimproveorigin1129fanhuidan_adv.pkl')

torch.save(predAnoattack,'./savedata/predAnoattack0103.pkl')

torch.save(predAattack,'./savedata/predAattack0103.pkl')

torch.save(targetAnoattack,'./savedata/targetAnoattack0103-0102.pkl')




#Step 5: Testing unknown attacks using DNN+CL models (after adversarial training)

all_correct_num = 0
all_sample_num = 0
model.eval()
for idx, (test_x, test_label) in enumerate(test_loader_D):
    
    test_x=test_x.cuda()
    test_label=test_label.cuda()
    feature1,predict_y = model(test_x.float().view(-1, 28*28),trainT=True)
    featestDzy+=feature1
    predict_y=predict_y.detach()
    predict_y=predict_y.cpu()
    predict_y = np.argmax(predict_y, axis=-1)
    predctDzy+=predict_y
    current_correct_num = predict_y == test_label.cpu()
    all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
    all_sample_num += current_correct_num.shape[0]
acc = all_correct_num / all_sample_num

#Step 5 end




preadzyori=predctAzyori+predctDzy

preadzyadv=predctAzyadv+predctDzy

featestzyori=featestAzyori+featestDzy

featestzyadv=featestAzyadv+featestDzy


#-----adv

torch.save(testAyizhiadv,'./savedata/testAyizhiadv0106.pkl')

#------pred
torch.save(predctAzyori,'./savedata/predctAzyori0106.pkl')

torch.save(predctAzyadv,'./savedata/predctAzyadv0106.pkl')

torch.save(predctDzy,'./savedata/predctDzy0106.pkl')

torch.save(preadzyori,'./savedata/preadzyori0106.pkl')

torch.save(preadzyadv,'./savedata/preadzyadv0106.pkl')

#------feat
torch.save(featestAzyori,'./savedata/featestAzyori0106.pkl')

torch.save(featestAzyadv,'./savedata/featestAzyadv0106.pkl')

torch.save(featestDzy,'./savedata/featestDzy0106.pkl')

torch.save(featestzyori,'./savedata/featestzyori0106.pkl')

torch.save(featestzyadv,'./savedata/featestzyadv0106.pkl')


#------train
torch.save(featesttrain,'./savedata/featesttrain1229.pkl')

epoch: 0 Time:1.486851453781128 clean_test_acc: 0.978  adv_test_acc: 0.914
----epoch: 0 Time:5.0408830642700195 clean_train_acc: 0.962  adv_train_acc: 0.937
epoch: 1 Time:1.1282050609588623 clean_test_acc: 0.970  adv_test_acc: 0.946
----epoch: 1 Time:5.366103172302246 clean_train_acc: 0.965  adv_train_acc: 0.940
epoch: 2 Time:1.3761327266693115 clean_test_acc: 0.974  adv_test_acc: 0.950
